In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os

# --- [1] 경로 및 파일 설정 (사용자 제공 파일명 반영) ---
DATA_PATH = "../../data/raw/"
OOF_PATH = "./oof_data/"
SUB_PATH = "./submissions/"

# 모델별 OOF 파일 (Large=Voted 방향인 m1, m7 vs Reverse인 m8)
M1_OOF = "exp11_sota_AUC_0.77212.npy"
M7_OOF = "exp25_m7_Refined_AUC_0.77375.npy"
M8_OOF = "exp28_m8_ft_AUC_0.76664.npy"

# 모델별 Submission 파일 (Small=Voted 방향)
M1_SUB = "0130-1923.csv"
M7_SUB = "m7_Refined_test_preds.csv"
M8_SUB = "28_m8_ft_0.76664.csv"

# --- [2] 데이터 로드 및 인덱스 정합성 확보 ---
train_df = pd.read_csv(f"{DATA_PATH}train.csv")

# m1 인덱스 (familysize <= 50)
m1_idx = train_df[train_df.familysize <= 50].index

# m7, m8 인덱스 (m7 필터링 기준: familysize <= 50 & std > 0)
a_cols = [f'Q{i}A' for i in 'abcdefghijklmnopqrst']
m7_mask = (train_df.familysize <= 50) & (train_df[a_cols].std(axis=1) > 0)
m7_idx = train_df[m7_mask].index

# 정답셋 추출 (방향: Large=Voted)
train_y = (2 - train_df.loc[m7_idx, 'voted'].to_numpy()).astype(np.float32)

# --- [3] OOF 로드 및 방향 보정 (Flip m8 Rank) ---
# m1 OOF: m7 인덱스에 맞춰 재정렬
m1_oof_raw = np.load(f"{OOF_PATH}{M1_OOF}")
m1_oof_df = pd.DataFrame(m1_oof_raw, index=m1_idx, columns=['val'])
m1_oof_aligned = m1_oof_df.reindex(m7_idx)['val'].values

# m7, m8 OOF 로드
m7_oof_aligned = np.load(f"{OOF_PATH}{M7_OOF}")
m8_oof_raw = np.load(f"{OOF_PATH}{M8_OOF}")

# Rank 변환 및 m8 방향 반전
m1_rank = pd.Series(m1_oof_aligned).rank(pct=True).values
m7_rank = pd.Series(m7_oof_aligned).rank(pct=True).values
# m8은 음의 상관관계가 확인되었으므로 1에서 빼서 방향을 뒤집음
m8_rank = 1 - pd.Series(m8_oof_raw).rank(pct=True).values 

# 3종 모델 균등 결합 (1:1:1)
ensemble_oof = (m1_rank + m7_rank + m8_rank) / 3
ensemble_auc = roc_auc_score(train_y, ensemble_oof)

print("-" * 50)
print(f"exp29_v2 (Corrected) Result")
print(f"Local AUC: {ensemble_auc:.5f}")
# 보정 후 상관계수가 양수(+)로 전환되었는지 확인
print(f"New Correlation (m1-m8): {np.corrcoef(m1_rank, m8_rank)[0,1]:.4f}")
print(f"New Correlation (m7-m8): {np.corrcoef(m7_rank, m8_rank)[0,1]:.4f}")
print("-" * 50)

# --- [4] 테스트 세트 앙상블 및 제출 파일 생성 ---
# 제출용 파일 로드 (Small=Voted 방향을 Large=Voted로 임시 변환)
m1_test = 2.0 - pd.read_csv(f"{SUB_PATH}{M1_SUB}")['voted']
m7_test = 2.0 - pd.read_csv(f"{SUB_PATH}{M7_SUB}")['voted']
m8_test = 2.0 - pd.read_csv(f"{SUB_PATH}{M8_SUB}")['voted']

# 테스트 데이터 순위 변환 및 m8 반전
m1_t_rank = m1_test.rank(pct=True).values
m7_t_rank = m7_test.rank(pct=True).values
m8_t_rank = 1 - m8_test.rank(pct=True).values 

final_rank = (m1_t_rank + m7_t_rank + m8_t_rank) / 3

# 최종 제출 파일 저장 (방향: Small=Voted 복구)
sub = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
sub['voted'] = 2.0 - final_rank
output_name = f"{SUB_PATH}exp29_v2_Rank_{ensemble_auc:.5f}.csv"
sub.to_csv(output_name, index=False)

print(f"Submission saved: {output_name}")

--------------------------------------------------
exp29_v2 (Corrected) Result
Local AUC: 0.77387
New Correlation (m1-m8): 0.9363
New Correlation (m7-m8): 0.9442
--------------------------------------------------
Submission saved: ./submissions/exp29_v2_Rank_0.77387.csv
